In [ ]:
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [ ]:
!pip install unsloth


In [ ]:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl==0.15.2 triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

In [ ]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.6.0+cu124
True
NVIDIA A30


In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 4096 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/home/administrator/VanIT/venVan/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.4.7: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    NVIDIA A30. Num GPUs = 2. Max memory: 23.486 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.4.7 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [ ]:
import json
from datasets import load_dataset
from tqdm import tqdm

dataset = load_dataset("thailevann/pretrain_dvcqg", split="train", streaming=True)
EOS_TOKEN = tokenizer.eos_token

with open("formatted_dataset.jsonl", "w", encoding="utf-8") as f:
    for example in tqdm(dataset, desc="Processing"):
        text = example["text"] + EOS_TOKEN
        json.dump({"text": text}, f, ensure_ascii=False)
        f.write("\n")

from datasets import load_dataset

dataset = load_dataset("json", data_files="formatted_dataset.jsonl", split="train")

Processing: 18903it [00:03, 4793.86it/s]
Generating train split: 18903 examples [00:00, 172688.53 examples/s]


In [ ]:
len(dataset)

18903

In [ ]:
# Tiền xử lý dữ liệu test
#dataset_test = load_dataset("thailevann/GovServices_Pretrain_Text_v2", split="test")



In [ ]:
for row in dataset[:5]["text"]:
    print("=========================")
    print(row)

**Tên thủ tục**: Đăng ký lần đầu đối với tổ chức hỗ trợ nghiên cứu có hoạt động phân tích thống kê và quản lý dữ liệu nghiên cứu thử nghiệm lâm sàng
**Cơ quan công bố**: Bộ Y tế
**Lĩnh vực**: Khoa học, công nghệ và đào tạo
**Cơ quan thực hiện**: Cục Khoa học Công nghệ và Đào tạo - Bộ Y tế; Cục Khoa học Công nghệ và Đào tạo - Bộ Y tế

**Trình tự thực hiện**:
Tổ chức đăng ký hoạt động giám sát nghiên cứu thử nghiệm lâm sàng nộp hồ sơ về Bộ Y tế.
Khi nhận hồ sơ, cơ quan tiếp nhận hồ sơ gửi cho tổ chức phiếu tiếp nhận hồ sơ theo mẫu 04 kèm theo Thông tư này, hướng dẫn bổ sung hồ sơ chưa hợp lệ.
Trong thời hạn 20 ngày làm việc kể từ ngày nhận đủ hồ sơ hợp lệ, Bộ Y tế (Cục Khoa học công nghệ và Đào tạo) có trách nhiệm xem xét, có văn bản chấp thuận hoạt động hỗ trợ nghiên cứu thử nghiệm lâm sàng. Trong trường hợp không chấp thuận, Bộ Y tế (Cục Khoa học công nghệ và Đào tạo) phải có thông báo bằng văn bản trong đó nêu rõ lý do không chấp thuận.<|im_end|>
**Tên thủ tục**: Thủ tục cấp lại Giấy 

In [ ]:
pip install wandb

In [ ]:
import wandb
wandb.login(key="YOUR_KEY")


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/administrator/.netrc
wandb: Currently logged in as: vanlethai12042002 (vanlethai12042002-ton-duc-thang-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
# Use wandb
import wandb
wandb.init(
    project="Chatbot-dvc",
    name="Qwen3-4b-CT-dvcqg-v1"
)


In [ ]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,

    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 16,

        num_train_epochs = 2,
        learning_rate = 1e-5,
        embedding_learning_rate = 2e-6,
        warmup_ratio = 0.1,
        lr_scheduler_type = "linear",
        #fp16 = not is_bfloat16_supported(),
        #bf16 = is_bfloat16_supported(),
        bf16 = True,
        fp16 = False,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        seed = 3407,
        output_dir = "outputs",
        report_to = "wandb",
    ),
)


Unsloth: Tokenizing ["text"] (num_proc=2): 100%|█| 18903/18903 [00:10<00:00, 178


In [ ]:
trainer_stats = trainer.train()
from huggingface_hub import login

login(token="YOUR_KEY")
model.push_to_hub("thailevann/Qwen3-4B_model_CT_DVCQG_v1")
tokenizer.push_to_hub("thailevann/Qwen3-4B_model_CT_DVCQG_v1")

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 18,903 | Num Epochs = 2 | Total steps = 294
O^O/ \_/ \    Batch size per device = 8 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (8 x 16 x 1) = 128
 "-____-"     Trainable parameters = 1,042,153,472/4,000,000,000 (26.05% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
10,1.529700
20,1.408500
30,1.214100
40,1.096700
50,1.040700
60,0.989400
70,0.963400
80,0.958600
90,0.919000
100,0.887300


100%|████████████████████████████████████████████| 1/1 [03:29<00:00, 209.52s/it]


Saved model to https://huggingface.co/thailevann/Qwen3-4B_model_CT_DVCQG_v1


  0%|                                                     | 0/1 [00:00<?, ?it/s]
tokenizer.json:   0%|                               | 0.00/11.4M [00:00<?, ?B/s]
tokenizer.json:  14%|███                   | 1.62M/11.4M [00:00<00:00, 11.6MB/s]
tokenizer.json:  24%|█████▎                | 2.79M/11.4M [00:01<00:04, 2.15MB/s]
tokenizer.json:  38%|████████▎             | 4.31M/11.4M [00:01<00:02, 3.09MB/s]
tokenizer.json:  53%|███████████▋          | 6.05M/11.4M [00:01<00:01, 4.02MB/s]
tokenizer.json:  73%|████████████████      | 8.36M/11.4M [00:01<00:00, 5.25MB/s]
tokenizer.json: 100%|██████████████████████| 11.4M/11.4M [00:02<00:00, 4.41MB/s]
100%|█████████████████████████████████████████████| 1/1 [00:02<00:00,  2.89s/it]
